This script turns the files downloaded from Refdata into our database that contains the leaflets

drugdb.db contains 3 tables :
- gtin_lang_lookup : query with gtin number and lang code ('fr', 'de' or 'it') to get the index of the leaflet content 
- drug_leaflets : query with index to get the content as a Jason object already structured into titles and paragraphs
- packages_info : contains additional infos for each leaflet (query with index to get all info)


How to use : update the paths to the documents in the first cell and run the notebook at once to create/update drugdb.db in the same directory as this file

In [12]:
### paths to update ###

path_SAI_Export = 'SAI-Export_20230405-030107' # relative path to zipped SAI-Export folder, remove .zip from the string
path_AipsDownload = 'AipsDownload_20230427' # relative path to zipped AipsDownload folder, remove .zip from the string

### end of paths to update ###

In [13]:
import zipfile
import os 
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

Get raw tables from Zip downloads

In [14]:
path_packages_data = 'SAI/SAI-Packungen.XML'
path_zip = path_SAI_Export + '.zip'
with zipfile.ZipFile(path_zip, 'r') as zip_ref:
    zip_ref.extract(path_packages_data, path_SAI_Export)

# read SAI-Packungen.XML with pd.read_xml
packages = pd.read_xml(path_SAI_Export + '/' + path_packages_data)
packages.dropna(subset=["GTIN_INDUSTRY"], inplace = True) 
packages.head()

,ZULASSUNGSNUMMER,SEQUENZNUMMER,PACKUNGSCODE,ZULASSUNGSSTATUS,BEMERKUNG_FREITEXT,PACKUNGSGROESSE,PACKUNGSEINHEIT,WIDERRUF_VERZICHT_DATUM,BTM_CODE,GTIN_INDUSTRY,IM_HANDEL_DATUM_INDUSTRY,AUSSER_HANDEL_DATUM_INDUSTRY,BESCHREIBUNG_DE_REFDATA,BESCHREIBUNG_FR_REFDATA
0,56775,1,2,D,Tube applicateur,5,G,2010-09-27,None,7.680568e+12,None,2010-09-27,GYNAZOL 1 Vag Cr 5 g,GYNAZOLE 1 cr vag 5 g
1,52203,1,15,D,None,6 x 10,G,2016-05-30,None,7.680522e+12,None,2016-05-30,NORGALAX Minieinlauf 6 x 10 g,NORGALAX lavement mini 6 x 10 g
5,53869,1,17,Z,None,20,SAC,None,None,7.680539e+12,None,None,MOVICOL Plv Btl 20 Stk,MOVICOL pdr sach 20 pce
6,53869,1,41,Z,None,100,SAC,None,None,7.680539e+12,None,None,MOVICOL Plv Btl 100 Stk,MOVICOL pdr sach 100 pce
9,57900,1,1,Z,None,2 + 2,SAC,None,None,7.680579e+12,None,None,MOVIPREP eine Anwendung Plv A+B Dppl Btl 2 Stk,MOVIPREP 1 application pdr A+B sach dbl 2 pce


Create 'packages_info'

In [15]:
packages_generated = packages.copy()
packages_generated.rename(columns={'GTIN_INDUSTRY': 'gtin'}, inplace=True)
packages_generated.rename(columns={'BESCHREIBUNG_FR_REFDATA': 'name'}, inplace=True)
packages_generated["gtin"] = packages_generated["gtin"].astype(int)

Parse leaflet

In [16]:
# extract AipsDownload_20230427\AipsDownload_20230427.xml from AipsDownload_20230427.zip
import zipfile
import os
import xml.etree.ElementTree as ET

path_zip = path_AipsDownload + '.zip'
with zipfile.ZipFile(path_zip, 'r') as zip_ref:
    zip_ref.extract(path_AipsDownload + '.xml', path_AipsDownload)

# parse the xml file
tree = ET.parse(path_AipsDownload + '/' + path_AipsDownload + '.xml')

# get the root element of the xml file
root = tree.getroot()

In [5]:
from bs4 import BeautifulSoup

def extract_content(html_content):
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all the paragraphs
    paragraphs = soup.find_all('p')

    # Initialize return variable
    content = []

    # Initialize variables
    title = ""
    current_paragraphs = []

    # Iterate over the paragraphs
    for paragraph in paragraphs:
        if paragraph.get('id'):
            # New title found
            if title:
                # Save the previous title and paragraphs
                content.append({'title': title, 'paragraphs': current_paragraphs})

            # Update the current title
            title = paragraph.get_text(strip=True)
            current_paragraphs = []
        else:
            # Collect paragraphs
            paragraph_text = paragraph.get_text(strip=True)
            if paragraph_text:
                current_paragraphs.append(paragraph_text)

    # Save the last title and paragraphs
    if title:
        content.append({'title': title, 'paragraphs': current_paragraphs})

    # delete "information to patient blabla" to get the name of the medicine to be the first element of the list
    # test if content is not empty
    if content:
        first_title = content[0]['title']
        # if first_title starts with 'Information' or 'Information' or 'Informazione', remove content[0]
        if first_title.startswith('Information') or first_title.startswith('Information') or first_title.startswith('Informazione'):
            content.pop(0)
    
    return content

In [6]:
drugs = []
df_drugs = pd.DataFrame(columns=['authNrs', 'content', 'lang'])

for i in range(len(root)):
    
    # init drug attributes
    authNrs = None
    atcCode = None
    content = None

    for j in range(len(root[i])):

        # get tag and text of the current element
        tag = root[i][j].tag
        text = root[i][j].text

        if tag == 'atcCode':
            atcCode = text
        elif tag == 'authNrs':
            authNrs = text
        elif tag == 'content':
            content_raw = text
    
    # if atcCode != None, don't append to dataframe
    # because only drugs with no atcCode are relevant to patients
    if atcCode == None:

        # get language string (fr, de, it, en)
        d = dict(root[i].attrib)
        lang = d['lang']

        content_extracted = extract_content(content_raw)

        # create drug dataframe
        df_drugs.loc[i] = [authNrs, content_extracted, lang]

# delete rows with empty content
df_drugs = df_drugs[df_drugs['content'].apply(lambda x: len(x) != 0)]
df_drugs.head()

c:\Users\thiba\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,authNrs,content,lang
52,55378,"[{'title': 'ACTOS', 'paragraphs': []}, {'title...",de
936,62684,"[{'title': 'Constella®', 'paragraphs': ['AbbVi...",de
1078,67667,"[{'title': 'Defitelio', 'paragraphs': ['DRAC A...",de
1327,65843,"[{'title': 'ELOCTA', 'paragraphs': []}, {'titl...",de
1997,52813,"[{'title': 'Iopidine®0,5%', 'paragraphs': []},...",de


Create 'gtin_lang_lookup' from df_drugs & packages

In [7]:
# keep only columns 'authNrs', 'lang'
authNrs = pd.DataFrame(df_drugs[['authNrs', 'lang']])
# index column becomes a column
authNrs.reset_index(inplace=True)
# split authNrs column by comma and space
authNrs['authNrs'] = authNrs['authNrs'].str.split(', ')
# explode authNrs column to make all combinations
authNrs = authNrs.explode('authNrs').reset_index(drop=True)
# rename columns "index" --> "ind" and "authNrs" --> "ZULASSUNGSNUMMER"
authNrs.rename(columns={'index': 'ind', 'authNrs': 'ZULASSUNGSNUMMER'}, inplace=True)
# convert ZULASSUNGSNUMMER to int
authNrs['ZULASSUNGSNUMMER'] = authNrs['ZULASSUNGSNUMMER'].astype(int)
authNrs

,ind,ZULASSUNGSNUMMER,lang
0,52,55378,de
1,936,62684,de
2,1078,67667,de
3,1327,65843,de
4,1997,52813,de
...,...,...,...
14680,25817,48143,it
14681,25817,52700,it
14682,25818,66431,it
14683,25819,55558,it


In [8]:
packages = packages[['GTIN_INDUSTRY', 'ZULASSUNGSNUMMER']]
packages['GTIN_INDUSTRY'] = packages['GTIN_INDUSTRY'].astype('Int64')
packages

,GTIN_INDUSTRY,ZULASSUNGSNUMMER
0,7680567750025,56775
1,7680522030155,52203
5,7680538690176,53869
6,7680538690411,53869
9,7680579000019,57900
...,...,...
58712,7680688420050,68842
58713,7680688420067,68842
58714,7680688530018,68853
58730,7680690070014,69007


In [9]:
# to each row of packages, add the ind and lang of the corresponding ZULASSUNGSNUMMER column
packages = packages.merge(authNrs, on='ZULASSUNGSNUMMER', how='left')
# drop if ind is NaN
packages.dropna(subset=['ind'], inplace=True)
# rename GTIN_INDUSTRY --> GTIN and drop ZULASSUNGSNUMMER
packages.rename(columns={'GTIN_INDUSTRY': 'GTIN'}, inplace=True)
packages.drop(columns=['ZULASSUNGSNUMMER'], inplace=True)
# ind type to int
packages['ind'] = packages['ind'].astype(int)
packages

,GTIN,ind,lang
2,7680538690176,14834,de
3,7680538690176,19338,fr
4,7680538690176,23843,it
5,7680538690411,14834,de
6,7680538690411,19338,fr
...,...,...,...
58027,7680688420067,20526,fr
58028,7680688420067,25021,it
58029,7680688530018,14940,de
58030,7680688530018,19444,fr


Create 'drug_leaflets' from df_drugs (clean df_drugs)

In [10]:
# drop authNrs column bcs we use index to locate now
df_drugs.drop(columns=['authNrs'], inplace=True)
df_drugs.drop(columns=['lang'], inplace=True)
# make the index a column
df_drugs.reset_index(inplace=True)
# rename index --> ind
df_drugs.rename(columns={'index': 'id'}, inplace=True)
df_drugs

,id,content
0,52,"[{'title': 'ACTOS', 'paragraphs': []}, {'title..."
1,936,"[{'title': 'Constella®', 'paragraphs': ['AbbVi..."
2,1078,"[{'title': 'Defitelio', 'paragraphs': ['DRAC A..."
3,1327,"[{'title': 'ELOCTA', 'paragraphs': []}, {'titl..."
4,1997,"[{'title': 'Iopidine®0,5%', 'paragraphs': []},..."
...,...,...
13522,25815,"[{'title': 'Zyloric®', 'paragraphs': ['100 mg ..."
13523,25816,[{'title': 'Zyprexa®compresse rivestite con fi...
13524,25817,"[{'title': 'Zyrtec®', 'paragraphs': ['UCB-Phar..."
13525,25818,"[{'title': 'Zytiga®, Compresse rivestite con f..."


Create drugdb.db that contains 'drug_leaflets', 'gtin_lang_lookup' and 'packages_info'

In [11]:
con = sqlite3.connect('drugdb.db')
    
# write dataframes to sqlite
packages_generated.to_sql('packages_info', con, if_exists='replace', index=False)
packages.to_sql('gtin_lang_lookup', con, if_exists='replace', index=False)

# write drug_df to sqlite with drug_df['content'] to json first
df_drugs['content'] = df_drugs['content'].apply(lambda x: pd.Series(x).to_json(orient='values'))
df_drugs.to_sql('drug_leaflets', con, if_exists='replace', index=False)

con.close()
